In [39]:
from utils.FEniCSx_solver import FEniCSx_solver
from dolfinx import mesh as msh, fem, default_scalar_type, la
from mpi4py import MPI
import ufl
import numpy as np

import scipy

nx=32
ny=32
comm = MPI.COMM_WORLD
cell_type=msh.CellType.triangle
mesh = msh.create_unit_square(comm=comm, nx=nx, ny=ny, cell_type=cell_type)

Wh = fem.functionspace(mesh, ('P', 1))
uh = fem.Function(Wh)
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
boundary_facets = msh.exterior_facet_indices(mesh.topology)
boundary_dofs = fem.locate_dofs_topological(Wh, mesh.topology.dim-1, boundary_facets)
eps = fem.Constant(mesh, default_scalar_type(1e-8))
b = ufl.as_vector((fem.Constant(mesh, default_scalar_type(1.0)),fem.Constant(mesh, default_scalar_type(0.0))))
c = fem.Constant(mesh, default_scalar_type(0.0))
f = fem.Constant(mesh, default_scalar_type(1.0))
bcs = [fem.dirichletbc(fem.Constant(mesh, default_scalar_type(0.0)), boundary_dofs, Wh)]

pde_data = mesh,Wh,uh,eps,b,None,f,None,bcs



cid_lims = mesh.topology.index_map(2).local_range
marker_ids = np.arange(cid_lims[0], cid_lims[1])

for index in range(cid_lims[0],cid_lims[1]):
    if np.intersect1d(Wh.dofmap.cell_dofs(index), bcs[0].dof_indices()[0]).size > 0:
            marker_ids = marker_ids[marker_ids!=index]

marker = np.ones_like(marker_ids, dtype=np.int32)
cell_tag = msh.meshtags(mesh, mesh.topology.dim, marker_ids, marker)
dx = ufl.Measure("dx", domain=mesh, subdomain_data=cell_tag, subdomain_id=1)

x = ufl.SpatialCoordinate(mesh)
ex_exp = x[0]*(1-ufl.exp(-(1-x[0])/eps))* (1 - (((ufl.exp(-(1-x[1])/eps)  + ufl.exp(-(x[1])/eps)))- ufl.exp(-(1)/eps))/(1-ufl.exp(-1/eps)))

exp = fem.Expression(ex_exp, Wh.element.interpolation_points())

u_ex = fem.Function(Wh)
u_ex.interpolate(exp)


residual = (-eps*ufl.div(ufl.grad(uh)) + ufl.dot(b, ufl.grad(uh)) + c * uh - f)**2 * dx

b_perp = ufl.as_vector((fem.Constant(mesh, default_scalar_type(0.0)),fem.Constant(mesh, default_scalar_type(-1.0))))
cross = abs(ufl.dot(b_perp, ufl.grad(uh)))
crosswind_loss = ufl.conditional(ufl.lt(cross, 1), 1/2*(5*cross**2 - 3*cross**3), ufl.sqrt(cross)) * dx
#loss = (uh-u_ex)**2 * ufl.dx
loss = residual + crosswind_loss
fs = FEniCSx_solver(pde_data=pde_data, loss_form=loss)


norm_b = ufl.sqrt(ufl.dot(b,b))
h = ufl.CellDiameter(domain=mesh) 
alpha = norm_b*h/(2*eps)
Xi = (1/ufl.tanh(alpha)-1/alpha)
tau_K = h/(2*norm_b)*Xi
Th = fem.functionspace(mesh, ('DG', 0))
tau = fem.Function(Th)
tau_exp = fem.Expression(tau_K, Th.element.interpolation_points())
tau.interpolate(tau_exp)
fs.set_weights(tau.x.array)


arr = np.array([])


In [40]:
from scipy.optimize import minimize
    

def callback(intermediate_result):
    fval = intermediate_result.fun
    print(f"J: {fval}")

def eval(weights):
    fs.set_weights(weights=weights)
    return fs.loss()

def eval_grad(weights):
    return fs.grad()


minimize(
    fun=eval,
    x0=fs.yh.x.array,
    jac=eval_grad,
    method='BFGS',
    tol=1e-9,
    options={"disp": True},
    callback=callback,
)

J: 0.34643899789798444
J: 0.3159638103585675
J: 0.3112069895600788
J: 0.306772210325142
J: 0.3014723671964748
J: 0.2940961699077968
J: 0.2867748531938717
J: 0.2836615000228563
J: 0.2802251850408381
J: 0.2779025430049525
J: 0.2746064829680037
J: 0.2691461492126031
J: 0.26156398653662066
J: 0.25124661853795643
J: 0.24454092910238057
J: 0.24190773194561835
J: 0.24171382543754696
J: 0.24136831935576414
J: 0.24086227678138608
J: 0.2400681359593381
J: 0.2387426832864546
J: 0.23637652760885286
J: 0.23353771952928284
J: 0.23245150284963748
J: 0.23203640875233986
J: 0.23142704912331502
J: 0.23029862292884923
J: 0.228166619429716
J: 0.22686865093214126
J: 0.22621284476777082
J: 0.22525038465161448
J: 0.21818189235223767
J: 0.21416423399973386
J: 0.20965460111466028
J: 0.20811699714853152
J: 0.20557752965078221
J: 0.2023551619896104
J: 0.19893515665497624
J: 0.19380977238361036
J: 0.18664665185295626
J: 0.1840830233411791
J: 0.18022769161414756
J: 0.17456774961499924
J: 0.1670358205540405
J: 0.15

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 0.03702458105065265
        x: [ 2.210e-02  1.812e-02 ...  3.561e+00  2.210e-02]
      nit: 403
      jac: [ 0.000e+00 -1.622e+00 ... -1.940e-05  0.000e+00]
 hess_inv: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            [ 0.000e+00  5.001e-01 ...  2.718e-02  0.000e+00]
            ...
            [ 0.000e+00  8.900e-03 ... -5.435e+01  0.000e+00]
            [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
     nfev: 704
     njev: 692

In [41]:
from utils.plotter import fem_plotter_grid
import pyvista as pv

pv.global_theme.cmap = 'coolwarm'


residual = (-eps*ufl.div(ufl.grad(fs.uh)) + ufl.dot(b, ufl.grad(fs.uh)) + c * fs.uh - f)**2 * ufl.TestFunction(Wh) * dx

#b_perp = ufl.conditional(ufl.eq(ufl.inner(b, b),0), b, -ufl.perp(b)/ufl.sqrt(ufl.inner(b,b)))
cross = ufl.max_value(ufl.dot(b_perp, ufl.grad(fs.uh)), -ufl.dot(b_perp, ufl.grad(fs.uh)))
crosswind_loss = ufl.conditional(ufl.lt(cross, 1), 1/2*(5*cross**2 - 3*cross**3), ufl.sqrt(cross)) * ufl.TestFunction(Wh) * dx

local_loss = fem.assemble_vector(fem.form(residual+crosswind_loss)).array


grid = fem_plotter_grid(Wh)
import pyvista as pv

p1, p2 = pv.Plotter(), pv.Plotter()

grid.add_data(fs.uh)
p1.subplot(0,0)
p1.add_mesh(
    grid.grid.warp_by_scalar(), 
    show_edges=True
)


grid.add_data(fs.yh)
p2.add_mesh(
    grid.grid,
    show_edges=True
)
p2.camera_position = 'xy'
p1.show()
p2.show()

Widget(value='<iframe src="http://localhost:50595/index.html?ui=P_0x372688f50_30&reconnect=auto" class="pyvist…

Widget(value='<iframe src="http://localhost:50595/index.html?ui=P_0x37c044910_31&reconnect=auto" class="pyvist…

from dolfinx import io
with io.XDMFFile(MPI.COMM_WORLD, "solution_quads_16x16.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(fs.uh)


with io.XDMFFile(MPI.COMM_WORLD, "params_quads_16x16.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(fs.uh)